In [ ]:
%%time
# LSTM for weather forecasting and time series implementation with regression framing

import numpy
import matplotlib.pyplot as plt
from pandas import read_csv
import math
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [ ]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return numpy.array(dataX), numpy.array(dataY)

In [ ]:
# fix random seed for reproducibility
numpy.random.seed(7)


# load the dataset
dataframe = read_csv('data8.csv', usecols=[1], engine='python', skipfooter=3)
dataframe.fillna(method='bfill', inplace=True)
dataset = dataframe.values
dataset = dataset.astype('float32')
dataframe.iloc[850:900]

In [ ]:
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

In [ ]:
# split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

In [ ]:
# reshape into X=t and Y=t+1
look_back = 6
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [ ]:
trainX.shape

In [ ]:
# reshape input to be [samples, time steps, features]
trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [ ]:
trainX.shape

In [ ]:
# create and fit the LSTM network
epochs = 10

model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=epochs, batch_size=32, verbose=1)

In [ ]:
# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

In [ ]:
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainYinv = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testYinv = scaler.inverse_transform([testY])

In [ ]:
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainYinv[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testYinv[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

In [ ]:
# shift train predictions for plotting
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict

# shift test predictions for plotting
testPredictPlot = numpy.empty_like(dataset)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict

In [ ]:
# plot baseline and predictions
val_from = 0
val_to = -1

plt.plot(scaler.inverse_transform(dataset)[val_from:val_to],  label = 'Dataset')
plt.plot(trainPredictPlot[val_from:val_to],  label = 'Train')
plt.plot(testPredictPlot[val_from:val_to],  label = 'Test')
plt.xlabel('Time Range')
plt.ylabel('Wind Speed (Normalized)')
plt.legend()
plt.title('Model Output Validation')
plt.show()





# fbprophet_predictions

In [ ]:
import numpy
import matplotlib.pyplot as plt
from pandas import read_csv
import math
import pandas as pd

dataframe = read_csv('data8.csv', engine='python')
dataframe.head(10)

In [ ]:
col = dataframe.columns
# dataframe.index
col

In [ ]:
df1 = dataframe.iloc[:,[3,6,9,12,15,18,21,24,27,30,33,36]].mean(axis=1)
df2 = dataframe.iloc[:,[4,7,10,13,16,19,22,25,28,31,34,37]].mean(axis=1)
df3 = dataframe.iloc[:,[5,8,11,14,17,20,23,26,29,32,35,38]].mean(axis=1)
dataframe['Mean Wind speed'], dataframe['Mean Wind Dispersion'], dataframe['Mean Wind Direction'] = [df1, df2, df3]
df = dataframe.iloc[:].round({'Mean Wind Dispersion':2, 'Mean Wind speed':2, 'Mean Wind Direction':2})
df['Date/Time']= pd.to_datetime(df['Date/Time'])
df.set_index('Date/Time', inplace=True)
df.iloc[850:1000]
# df.columns

In [ ]:
# df4 = read_csv('data8.csv', usecols=[0,1], engine='python', skipfooter=1)
pd.DataFrame(df)
dfc1 = df[['Mean Wind speed','Mean Wind Dispersion','Mean Wind Direction']].copy()
dfc1.iloc[3800:4000]
dfc1

In [ ]:
# dfc1.datetime.rename('ds', inplace=False)
# dfc1.index.name='ds'
# dfc1.columns.name='y'
# dfc1.rename(columns={'Mean Wind speed':'y'}, inplace = True) 
# dfc1.rename(columns = {'Mean Wind speed':'y'}, inplace = True) 
# dfc2=dfc1.resample('H').mean()
dfc2=dfc1.copy()
dfc=dfc2.drop(columns=[ 'Mean Wind Dispersion','Mean Wind Direction'])
dfc.shape

In [ ]:
from fbprophet import Prophet

dataset=dfc.reset_index()[['Date/Time', 'Mean Wind speed']].rename({'Date/Time':'ds', 'Mean Wind speed':'y'}, axis='columns')
# split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size], dataset[train_size:len(dataset)]


In [ ]:
train.shape,test.shape

In [ ]:
m = Prophet(interval_width=0.95, yearly_seasonality=True, monthly_seasonality=True, daily_seasonality=True)
train
m.fit(train)  # dfc is a pandas.DataFrame with 'y' and 'ds' columns
# m.params

In [ ]:
future = m.make_future_dataframe(periods=144, freq='10min')
forecast=m.predict(future)
fbp = forecast['yhat']


In [ ]:
forecast.tail()

In [ ]:
m.plot(forecast)

In [ ]:
pd.concat([train.set_index('ds')['y'], forecast.set_index('ds')['yhat']],axis=1).plot()

In [ ]:
m.plot_components(forecast)

In [ ]:
from fbprophet.plot import add_changepoints_to_plot

fig = m.plot(forecast)
add_changepoints_to_plot(fig.gca(), m, forecast)

In [ ]:
m.changepoints

In [ ]:
import plotly.graph_objs as go

In [ ]:
from fbprophet.plot import plot_plotly
import plotly.offline as ply

fig = plot_plotly(m, forecast)  #this returns a plotly figure
ply.iplot(fig)

In [ ]:
from fbprophet.diagnostics import cross_validation as cv
results = cv(model=m, initial = '395days', horizon = '195 days')
results.head()

In [ ]:
from fbprophet.diagnostics import performance_metrics as pm

pm(cv)

In [ ]:
dfcc=dfc.interpolate(model='linear')
dataset = dfc.fillna(dfcc)
dataset.iloc[850:900]

In [ ]:
m=prophet()
m.fit(df)

prophet_forecast = prophet_model.make_future_dataframe(periods=7, freq="d")
prophet_forecast = prophet_model.predict(prophet_forecast)

fig1 = prophet_model.plot(prophet_forecast)

datenow = datetime.now()
dateend = datenow + timedelta(days=7)
datestart = dateend - timedelta(days=14)

plt.xlim([datestart, dateend])
plt.title("Email forecast", fontsize=20)
plt.xlabel("Day", fontsize=20)
plt.ylabel("Emails expected", fontsize=20)
plt.axvline(datenow, color="k", linestyle=":")
plt.show()